In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dsj1000/dsj1000.tsp
/kaggle/input/tsp532/att532.tsp
/kaggle/input/data1291/d1291.tsp


In [4]:
def read_cities(filename):
    with open(filename, 'r') as f:
        # Skip the header lines
        for i in range(6):
            next(f)
        
        # Extract the coordinates and build the tuples
        cities = []
        for line in f:
            if line.startswith('EOF'):
                break
            values = line.split()
            if len(values) >= 3:
                x, y = values[1], values[2]
                cities.append((float(x), float(y)))
    return cities

In [2]:
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
len(cities)

532

In [5]:
import numpy as np
import pandas as pd
import random
import time

# Define the fitness function
def fitness(individual, distances):
    distance = 0
    for i in range(len(individual)-1):
        distance += distances[individual[i], individual[i+1]]
    distance += distances[individual[-1], individual[0]]
    return distance

# Define the crossover function
def crossover(parent1, parent2):
    child = [-1] * len(parent1)
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    for i in range(startGene, endGene):
        child[i] = parent1[i]
    for i in range(len(parent2)):
        if parent2[i] not in child:
            for j in range(len(child)):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

# Define the mutation function
def mutation(individual):
    geneA = int(random.random() * len(individual))
    geneB = int(random.random() * len(individual))
    individual[geneA], individual[geneB] = individual[geneB], individual[geneA]
    return individual

# Define the genetic algorithm function
def genetic_algorithm(distances, population_size, generations=100):
    # Create the initial population
    population = []
    for i in range(population_size):
        individual = list(range(len(distances)))
        random.shuffle(individual)
        population.append(individual)
    
    # Evolve the population
    for generation in range(generations):
        # Evaluate the fitness of each individual
        fitness_values = []
        for individual in population:
            fitness_values.append(fitness(individual, distances))
        
        # Select the parents for the next generation
        parents = []
        for i in range(population_size):
            parent1 = population[random.randint(0, population_size-1)]
            parent2 = population[random.randint(0, population_size-1)]
            if fitness(parent1, distances) < fitness(parent2, distances):
                parents.append(parent1)
            else:
                parents.append(parent2)
        
        # Create the next generation
        new_population = []
        for i in range(population_size//2):
            parent1 = parents[i]
            parent2 = parents[population_size-i-1]
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)
            new_population.append(mutation(child1))
            new_population.append(mutation(child2))
        population = new_population        
        population_size = len(population)
    
    # Find the best individual
    fitness_values = []
    for individual in population:
        fitness_values.append(fitness(individual, distances))
    best_individual = population[np.argmin(fitness_values)]
    
    return best_individual

In [8]:
#512 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/tsp532/att532.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))

Best Individual: [260, 77, 150, 95, 185, 278, 411, 116, 182, 172, 88, 187, 6, 193, 93, 101, 53, 370, 468, 351, 241, 68, 430, 280, 189, 90, 274, 285, 451, 362, 484, 242, 486, 228, 243, 86, 337, 8, 134, 125, 63, 447, 9, 219, 413, 460, 501, 194, 303, 81, 196, 42, 73, 108, 173, 240, 156, 508, 531, 440, 136, 402, 244, 474, 516, 178, 49, 87, 414, 436, 475, 39, 418, 363, 322, 120, 114, 162, 398, 529, 522, 524, 490, 505, 289, 191, 275, 29, 309, 180, 117, 91, 168, 328, 476, 354, 218, 382, 485, 226, 383, 201, 520, 198, 445, 346, 324, 299, 366, 469, 276, 449, 446, 221, 481, 482, 442, 376, 507, 145, 209, 263, 428, 290, 202, 359, 184, 80, 318, 265, 256, 133, 284, 261, 498, 466, 302, 230, 503, 368, 473, 500, 526, 330, 506, 204, 31, 207, 331, 220, 246, 24, 314, 338, 390, 251, 18, 62, 294, 157, 98, 391, 0, 394, 369, 104, 74, 69, 483, 7, 319, 139, 344, 103, 215, 396, 25, 14, 361, 461, 511, 457, 521, 295, 223, 211, 463, 137, 353, 472, 3, 247, 222, 519, 513, 530, 441, 504, 97, 380, 327, 84, 206, 326, 118

In [10]:
#1000 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/dsj1000/dsj1000.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))

Best Individual: [736, 981, 653, 910, 332, 573, 311, 31, 761, 193, 868, 322, 497, 634, 700, 268, 416, 0, 919, 5, 980, 952, 426, 691, 645, 84, 900, 329, 481, 837, 649, 367, 804, 638, 581, 30, 85, 998, 105, 831, 379, 289, 344, 120, 342, 438, 776, 917, 521, 965, 307, 224, 662, 502, 984, 974, 264, 703, 678, 280, 820, 999, 989, 828, 116, 798, 411, 501, 937, 454, 258, 437, 708, 284, 902, 21, 214, 765, 842, 302, 488, 627, 352, 75, 290, 546, 915, 383, 319, 538, 431, 847, 656, 628, 784, 994, 889, 979, 672, 477, 447, 602, 103, 666, 898, 865, 547, 959, 575, 337, 694, 727, 584, 36, 298, 580, 161, 943, 796, 230, 767, 316, 450, 323, 955, 41, 370, 756, 492, 101, 57, 630, 402, 808, 257, 256, 870, 893, 275, 54, 871, 455, 867, 297, 971, 996, 439, 253, 715, 391, 464, 760, 674, 160, 396, 941, 782, 381, 874, 33, 480, 692, 475, 626, 855, 180, 241, 490, 812, 147, 333, 845, 651, 682, 530, 170, 816, 787, 966, 272, 699, 744, 194, 562, 314, 869, 920, 371, 614, 198, 223, 15, 330, 859, 883, 390, 520, 410, 399, 17,

In [6]:
#1291 cities
# Load the cities dataset
cities = read_cities("/kaggle/input/data1291/d1291.tsp")
distances = np.zeros((len(cities), len(cities)))
for i in range(len(cities)):
    for j in range(len(cities)):
        distances[i,j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

# Run the genetic algorithm
start_time = time.time()
best_individual = genetic_algorithm(distances, len(cities))
end_time = time.time()

# Print the best individual and its fitness value
print('Best Individual:', best_individual)
print('Fitness Value:', fitness(best_individual, distances))
print('Running Time:', end_time - start_time)
print('Size cities:', len(cities))

KeyboardInterrupt: 